In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
print(tf.__version__)

1.8.0


In [3]:
training_set_size_portion = 0.8
do_shuffle = True
accuracy_score = 0
hidden_units_spec = [10,20,10]
n_classes_spec = 2
steps_spec = 200000
epochs_spec = 15
tmp_dir_spec = "C:/Users/Kaiwalya/Desktop/ML/Source codes/NN/ckpt_tranfusion"
file_name = "transfusion.csv"
features = ['recency','tinv','monetary','t']
# Here's the label that we want to predict -- it's also a column in # the CSV
labels = ['op']


In [4]:

my_data = pd.read_csv("transfusion.csv",dtype=float)

In [5]:
my_data.head()

,recency,tinv,monetary,t,op
0,2.0,50.0,12500.0,98.0,1.0
1,0.0,13.0,3250.0,28.0,1.0
2,1.0,16.0,4000.0,35.0,1.0
3,2.0,20.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0


In [6]:
if do_shuffle:
 randomized_data = my_data.reindex(np.random.permutation(my_data.index))
else:
 randomized_data = my_data

In [7]:
my_data.head(20)

,recency,tinv,monetary,t,op
0,2.0,50.0,12500.0,98.0,1.0
1,0.0,13.0,3250.0,28.0,1.0
2,1.0,16.0,4000.0,35.0,1.0
3,2.0,20.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
5,4.0,4.0,1000.0,4.0,0.0
6,2.0,7.0,1750.0,14.0,1.0
7,1.0,12.0,3000.0,35.0,0.0
8,2.0,9.0,2250.0,22.0,1.0
9,5.0,46.0,11500.0,98.0,1.0


In [8]:
total_records = len(randomized_data)
training_set_size = int(total_records * training_set_size_portion)
test_set_size = total_records = training_set_size

In [9]:
# Build the training features and labels
training_features = randomized_data.head(training_set_size)[features].copy()
training_labels = randomized_data.head(training_set_size)[labels].copy()
print(training_features.head())
print(training_labels.head())

     recency  tinv  monetary     t
231      4.0   1.0     250.0   4.0
77       2.0   2.0     500.0   4.0
66       2.0   4.0    1000.0  14.0
117      4.0  11.0    2750.0  61.0
445     21.0   3.0     750.0  35.0
      op
231  1.0
77   0.0
66   1.0
117  0.0
445  0.0


In [10]:
# Build the testing features and labels
testing_features = randomized_data.tail(test_set_size)[features].copy()
testing_labels = randomized_data.tail(test_set_size)[labels].copy()

In [11]:
feature_columns = [tf.feature_column.numeric_column(key) for key in features]

In [12]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns, 
    hidden_units=hidden_units_spec, 
    n_classes=n_classes_spec,
    model_dir=tmp_dir_spec)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:/Users/Kaiwalya/Desktop/ML/Source codes/NN/ckpt_tranfusion', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023B3D9FCB00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
# Define the training input function
train_input_fn = tf.estimator.inputs.pandas_input_fn(x=training_features,
                                                     y=training_labels,
                                                     num_epochs=epochs_spec,
                                                     shuffle=True)

In [47]:
# Train the model using the classifer.
classifier.train(input_fn=train_input_fn, steps=steps_spec)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:/Users/Kaiwalya/Desktop/ML/Source codes/NN/ckpt_tranfusion\model.ckpt-2059
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2060 into C:/Users/Kaiwalya/Desktop/ML/Source codes/NN/ckpt_tranfusion\model.ckpt.
INFO:tensorflow:loss = 79.0026, step = 2060
INFO:tensorflow:Saving checkpoints for 2130 into C:/Users/Kaiwalya/Desktop/ML/Source codes/NN/ckpt_tranfusion\model.ckpt.
INFO:tensorflow:Loss for final step: 1.98219.


In [48]:
# Define the test input function
test_input_fn = tf.estimator.inputs.pandas_input_fn(
                    x=testing_features, 
                    y=testing_labels, 
                    num_epochs=epochs_spec, 
                    shuffle=False)

In [49]:
# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("Accuracy = {}".format(accuracy_score))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-27-19:49:24
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:/Users/Kaiwalya/Desktop/ML/Source codes/NN/ckpt_tranfusion\model.ckpt-2130
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-27-19:49:25
INFO:tensorflow:Saving dict for global step 2130: accuracy = 0.782609, accuracy_baseline = 0.767559, auc = 0.759667, auc_precision_recall = 0.514003, average_loss = 0.468371, global_step = 2130, label/mean = 0.232441, loss = 59.1731, precision = 0.569231, prediction/mean = 0.272771, recall = 0.266187
Accuracy = 0.782608687877655


In [50]:
# Create a prediction set -- this is a list of input features that you want to classify
prediction_set = pd.DataFrame({'recency':[2, 1], 'monetary':[4,3],'tinv':[3,2],'t':[5,4],'op':[4,5]})

In [33]:
predict_input_fn = tf.estimator.inputs.pandas_input_fn(
                      x=prediction_set, 
                      num_epochs=1, 
                      shuffle=False)

In [19]:
# Get a list of the predictions
predictions = list(classifier.predict(input_fn=predict_input_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:/Users/Kaiwalya/Desktop/ML/Source codes/NN/ckpt_tranfusion\model.ckpt-568
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [20]:
predicted_classes = [p["classes"] for p in predictions] 
results=np.concatenate(predicted_classes) 
print(results)

[b'0' b'0']
